# Capstone Project - The Battle of the Neighborhoods-Finding A Hot spot For Having Fun in Frankfurt am Main City 
#### Created By Setareh Behzadi Pishkenari
### Applied Data Science Capstone by IBM/Coursera

## 
## Table of contents
 

#### Introduction
#### Data
#### Methodology
#### Execution steps
#### Result
#### Conclusion
#### References

## Introduction: Business Problem
 
In this project will try to give advise and discuss about hot spot for have fun in Frankfurt am Main. Specifically, this report will be targeted to travelers and families or any other people interested in having Fun in Frankfurt am Main city, Germany that they have no idea for a optimal place with any sufficient time, for examplefor a day trip or a short trip. I will use my data science powers to generate a most promissing neighborhood based on criteria. A spot will then be clearly expressed so that best possible final Place can be chosen by travelers and families or any other people.

## Data 


First of all I need to collect Data of all Funny Places in Frankfurt am Main including their name, id, location (address, latitude, longitude) then pick up the "hot" neighbor where locates most of the venue. Number of existing Funny places in the neighborhood (any type such as Restaurants, Hotels, Shops, Caffee, Fitness, Games,...) and their type and location in every neighborhood will be obtained using Foursquare API and applied folium for visualizing a particular neighbor.


## Methodology
In this project I will direct my efforts on detecting a spot of Frankfurt am Main, Germany that have more Fun, particularly I will have not limit my analysis to access area around city.
In first step I have collected the required data: location and category of fun. I have also identified  data according to Foursquare categorization. Second step in my analysis will be calculation and exploration of Funny places across different areas of Frankfurt am Main. In third and final step I will focus on most promising spot by spotting the clusters of items we can see which neighborhood has density of Fun places, based on Foursqure Data.I will present map of all locations created by that using Heatmaps.

##  Execution steps

I import all the tools I need.


In [2]:

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
import sys
!pip install beautifulsoup4
from bs4 import BeautifulSoup
!pip install geocoder
import geocoder
import os
!pip install folium
import folium # map rendering library
!pip install geopy
from geopy.geocoders import Nominatim 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline


print('Libraries imported.')

     |████████████████████████████████| 93 kB 2.8 MB/s  eta 0:00:01
Libraries imported.


### Foursquare

Now let's Apply my credential ID on Foursquare. I will use Foursquare API to get info on Fun places in neighborhoods. We're interested in venues in 'Fun' category. 

In [3]:
CLIENT_ID = 'WASXSEP5X50WDZCDXA2LTZZKMU1ZOSHTX52MEKY030OZPD0Y' # your Foursquare ID
CLIENT_SECRET = 'XP0GIGTRP0I0IVZOTFCMWGPBIYZ5M1KCF5J1SHKM0ZG1YZIZ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WASXSEP5X50WDZCDXA2LTZZKMU1ZOSHTX52MEKY030OZPD0Y
CLIENT_SECRET:XP0GIGTRP0I0IVZOTFCMWGPBIYZ5M1KCF5J1SHKM0ZG1YZIZ


In [4]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "Fun",
    "near": "Frankfurt am Main",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [5]:
print(data)

<Response [200]>


Now I will Transform data into json then request geocode.


In [6]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])

In [7]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Frankfurt am Main', 'Frankfurt am Main')

In [8]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 50.12012425, 'lng': 8.692766964990234},
  'sw': {'lat': 50.10969075, 'lng': 8.673517735205078}},
 125)

In [9]:
d["geocode"]

{'what': '',
 'where': 'frankfurt am main',
 'center': {'lat': 50.11552, 'lng': 8.68417},
 'displayString': 'Frankfurt am Main, Germany',
 'cc': 'DE',
 'geometry': {'bounds': {'ne': {'lat': 50.22525399547315,
    'lng': 8.800204924159232},
   'sw': {'lat': 50.01235400018925, 'lng': 8.471343003421413}}},
 'slug': 'frankfurt-am-main-germany',
 'longId': '72057594040853469'}

I start creating group including information which is recommended.

In [10]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [11]:
d["groups"][0]["type"], d["groups"][0]["name"]


('Recommended Places', 'recommended')

Creating items of objects Fun and their attributes - id, address, name, etc

items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

## Results 

From the output I can identify necessary factors of what I will use later to consider the probability of launching our up-to-coming location. Based on that we start to organize what we have got.

In [13]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head(10)

{'address': 'Konstablerwache', 'crossStreet': 'Zeil', 'lat': 50.114516186984, 'lng': 8.68711555553734, 'labeledLatLngs': [{'label': 'display', 'lat': 50.114516186984, 'lng': 8.68711555553734}], 'postalCode': '60313', 'cc': 'DE', 'city': 'Frankfurt am Main', 'state': 'Hessen', 'country': 'Deutschland', 'formattedAddress': ['Konstablerwache (Zeil)', '60313 Frankfurt am Main', 'Deutschland']}
{'address': 'Zeil 105', 'lat': 50.114162979496456, 'lng': 8.682001296163765, 'labeledLatLngs': [{'label': 'display', 'lat': 50.114162979496456, 'lng': 8.682001296163765}], 'postalCode': '60313', 'cc': 'DE', 'city': 'Frankfurt am Main', 'state': 'Hessen', 'country': 'Deutschland', 'formattedAddress': ['Zeil 105', '60313 Frankfurt am Main', 'Deutschland']}
{'address': 'Schärfengäßchen 6', 'crossStreet': 'Holzgraben', 'lat': 50.11350906292216, 'lng': 8.68168610237595, 'labeledLatLngs': [{'label': 'display', 'lat': 50.11350906292216, 'lng': 8.68168610237595}], 'postalCode': '60311', 'cc': 'DE', 'neighbor

,uid,name,shortname,address,postalcode,lat,lng
0,4b058852f964a520bdbe22e3,Erzeugermarkt Konstablerwache,Farmer's Market,,60313,50.114516,8.687116
1,53e223cf498e2cf5740be372,FITSEVENELEVEN Zeil,Gym / Fitness,,60313,50.114163,8.682001
2,57b34f2e498edc52148534fb,Góc Phố,Vietnamese,,60311,50.113509,8.681686
3,4b058852f964a520bfbe22e3,Kleinmarkthalle,Market,,60311,50.112778,8.682958
4,4c012d6e9a950f476ca808c6,LEGO Store,Toys & Games,,60313,50.114708,8.681621
5,4b05884ef964a5206cbd22e3,Zum Schwejk,Bar,,60313,50.115885,8.684775
6,555731f6498e1fab8ac2ad9f,Fein,Café,,60313,50.118837,8.685420
7,4b058852f964a520d4be22e3,Peek & Cloppenburg,Apparel,,60313,50.114194,8.684896
8,4e11a3be1f6e671dd9e93729,Jumeirah Frankfurt,Hotel,,60313,50.114999,8.680512
9,4dbfe3314b2221ec2d638030,AROYDEE Thai Küche,Thai,,60313,50.116420,8.680245


As I can see that there are many Fun places without address. I need to execute hasattr() to determine if each object has a attribute (address). Next step I will execute a very important part - get coordinates of Frankfurt am Main and create folium map which will help visualize what I have got from data.

In [14]:
FrankfurtamMain_center = d["geocode"]["center"]
FrankfurtamMain_center

{'lat': 50.11552, 'lng': 8.68417}

In [15]:
from folium import plugins


map_FrankfurtamMain = folium.Map(location=[50.1109, 8.6821], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_FrankfurtamMain)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_FrankfurtamMain.add_child(plugins.HeatMap(hm_data))

map_FrankfurtamMain

Here is beautiful Frankfurt am Main with little red dots presenting different items. By spotting the clusters of items I can see which neighborhood has density of Fun places.

In [16]:
lat = 50.1109
lng = 8.6821
map_FrankfurtamMain = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="O Day Roi!",
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_FrankfurtamMain)
map_FrankfurtamMain

We will need a location where we can catch out optimal Fun place. Look at the blue buble, here we find out that it will recommend to go  about the crossroad on Paulsplatz street. This neighborhood is safe and right at the center to have a funny Plan. 

 ## Conclusion


Purpose of this project was to identify optimal Funny location in order to aid and give advise travelers and families or any other people interested in having Fun in Frankfurt am Main city, Germany for a short time. Final decission on optimal Fun location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like famous and attraction of each place, prices and etc.


## References 

#### [1] IBM/Coursera

#### [2] Foursquare